In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

In [2]:
dftrain=pd.read_csv('AAATrain.csv')

In [3]:
dftest=pd.read_csv('AAATest.csv')

In [113]:
model = sm.logit(formula='SoCal ~ travel_total_bookings + travel_num_cancelled_bookings \
        + travel_daysbetween_booking_depart + travel_revenue_index + travel_tripvalue_index + individual_age + \
        + individual_education + individual_race + individual_children_7_9 + individual_children_13_18 + Quartile_2 + individual_children_4_6 \
        + days_between + Quartile_3', data=dftrain).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.144615
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  SoCal   No. Observations:                45800
Model:                          Logit   Df Residuals:                    45773
Method:                           MLE   Df Model:                           26
Date:                Tue, 16 Jul 2024   Pseudo R-squ.:                 0.03007
Time:                        14:41:25   Log-Likelihood:                -6623.4
converged:                       True   LL-Null:                       -6828.7
Covariance Type:            nonrobust   LLR p-value:                 8.278e-71
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                               

In [115]:
Xtest = dftest[['travel_total_bookings', 'travel_num_cancelled_bookings', 'travel_daysbetween_booking_depart', \
                'travel_revenue_index', 'travel_tripvalue_index', 'individual_age', 'Quartile_2', \
                'individual_children_4_6', 'individual_education', 'individual_race', 'individual_children_7_9', 'individual_children_13_18', \
                'days_between', 'Quartile_3']]
ytest = dftest['SoCal']

In [127]:
SoCal_yhat = model.predict(Xtest)

In [128]:
dftest['SoCal_yhat'] = SoCal_yhat

In [129]:
print(SoCal_yhat)

33682    0.999863
26530    0.994075
21550    0.993514
3693     0.961334
20049    0.944860
           ...   
17604    0.000256
20168    0.000230
16519    0.000219
34308    0.000147
22252    0.000014
Length: 45800, dtype: float64


In [130]:
dftest = dftest.sort_values(by = 'SoCal_yhat', ascending = False)

In [131]:
dftest['SoCal_Quintile'] = pd.qcut(dftest['SoCal_yhat'], 5, labels=[i for i in range (5, 0, -1)])

In [132]:
print(dftest['SoCal_Quintile'])

33682    1
26530    1
21550    1
3693     1
20049    1
        ..
17604    5
20168    5
16519    5
34308    5
22252    5
Name: SoCal_Quintile, Length: 45800, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]


In [133]:
dftest['SoCal_Quintile'] = dftest['SoCal_Quintile'].astype('int')

In [136]:
SoCal_travelers = pd.crosstab(dftest['SoCal_Quintile'], dftest['SoCal'], normalize='index')

In [137]:
print(SoCal_travelers)

SoCal                  0         1
SoCal_Quintile                    
1               0.937445  0.062555
2               0.961354  0.038646
3               0.968450  0.031550
4               0.972707  0.027293
5               0.980240  0.019760


In [138]:
dftest.to_csv('SOCALfile.csv')